# QCoDeS Example with Ithaco

In [1]:
# %matplotlib nbagg
%gui qt 
import matplotlib.pyplot as plt
import time
import numpy as np

import qcodes as qc
from qcodes.utils.validators import Enum, Strings
import qcodes.instrument_drivers.tektronix.Keithley_2600_channels as keith
import qcodes.instrument_drivers.agilent.Agilent_34400A as agi
import qcodes.instrument_drivers.ithaco.Ithaco_1211 as ithaco

from qcodes.instrument.parameter import Parameter

from qcodes.plots.pyqtgraph import QtPlot
from qcodes.loops import Loop

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\Jens-Work\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\Jens-Work\.qcodes\logs\201006-17520-qcodes.log


In [2]:
import time
class Timer(object):
    def __init__(self, name=None):
        self.name = name

    def __enter__(self):
        self.tstart = time.time()

    def __exit__(self, type, value, traceback):
        if self.name:
            print('[%s]' % self.name,)
        print('Elapsed: %s' % (time.time() - self.tstart))

In [3]:
# create Instruments
k1 = keith.Keithley_2600('Keithley1', 'GPIB0::15::INSTR')

a1 = agi.Agilent_34400A('Agilent1', 'GPIB0::11::INSTR')
a2 = agi.Agilent_34400A('Agilent2', 'GPIB0::6::INSTR')

camp = ithaco.Ithaco_1211('camp1')
camp.sens.set(1e-4)

curr = ithaco.CurrentParameter(a2.volt, camp)

# set integration time (number of line cycles)
a1.NPLC.set(1)
a2.NPLC.set(1)

In [4]:
camp.snapshot()

{'functions': {},
 'metadata': {'info': {'model': '1211',
   'serial_number': None,
   'software_revision': None,
   'vendor': 'Ithaco (DL Instruments)'}},
 'parameters': {'invert': {'ts': '2016-04-21 15:56:06', 'value': True},
  'risetime': {'ts': '2016-04-21 15:56:06', 'value': 0.3},
  'sens': {'ts': '2016-04-21 15:56:06', 'value': 0.0001},
  'sens_x': {'ts': '2016-04-21 15:56:06', 'value': 1},
  'suppression': {'ts': '2016-04-21 15:56:06', 'value': 1e-07}}}

In [5]:
a2.volt.units

'V'

In [6]:
curr.units

('V', 'A')

In [7]:
data = Loop(k1.a.volt[-5:5:1], 0).each(curr).run(location='testsweep', overwrite=True)
plotQ = QtPlot(data.current,windowTitle='YEAH')

DataSet: DataMode.PULL_FROM_SERVER, location='testsweep'
   current: current
   volt: volt
   volt_set: volt
started at 2016-04-21 15:56:10


In [16]:
plotQ

In [9]:
data.sync()
data.arrays

{'current': DataArray[10]: current
 array([ -4.98941000e-04,  -3.99618000e-04,  -3.00370000e-04,
         -2.01084000e-04,  -1.01763000e-04,  -2.42637000e-06,
          9.69014000e-05,   1.96216000e-04,   2.95535000e-04,
          3.94851000e-04]), 'volt': DataArray[10]: volt
 array([ 4.98941  ,  3.99618  ,  3.0037   ,  2.01084  ,  1.01763  ,
         0.0242637, -0.969014 , -1.96216  , -2.95535  , -3.94851  ]), 'volt_set': DataArray[10]: volt_set
 array([-5., -4., -3., -2., -1.,  0.,  1.,  2.,  3.,  4.])}

In [10]:
station1 = qc.Station(a1,a2)
station1.set_measurement(a1.volt)
station2 = qc.Station(a1,a2)
station2.set_measurement(a1.volt, a2.volt)

In [11]:
# Time single readings
with Timer('Time s1'):
    station1.measure()
with Timer('Time s2'):
    station2.measure()

[Time s1]
Elapsed: 0.05600309371948242
[Time s2]
Elapsed: 0.10500597953796387


In [12]:
# Time single readings
with Timer('Time a1'):
    a1.volt.get()
with Timer('Time a2'):
    a2.volt.get()

[Time a1]
Elapsed: 0.05600309371948242
[Time a2]
Elapsed: 0.049002885818481445


In [13]:
with Timer('Time Loop 1'):
    data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt).run(location='testsweep', overwrite=True,background=False)

with Timer('Time Loop 2'):
    data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt, a2.volt).run(location='testsweep', overwrite=True,background=False)

DataSet: DataMode.PUSH_TO_SERVER, location='testsweep'
   volt_set: volt
   volt: volt
started at 2016-04-21 15:51:15
[Time Loop 1]
Elapsed: 0.5770328044891357
DataSet: DataMode.PUSH_TO_SERVER, location='testsweep'
   volt_set: volt
   volt_0: volt
   volt_1: volt
started at 2016-04-21 15:51:16
[Time Loop 2]
Elapsed: 1.0900623798370361


In [14]:
with Timer('Time Loop 1'):
    data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt).run(location='testsweep', overwrite=True)
    while data.sync():
        time.sleep(0.1)

DataSet: DataMode.PULL_FROM_SERVER, location='testsweep'
   volt_set: volt
   volt: volt
started at 2016-04-21 15:51:16
[Time Loop 1]
Elapsed: 1.075061559677124


In [15]:
with Timer('Time Loop 2'):
    data = Loop(k1.a.volt[-5:5:1], 0).each(a1.volt, a2.volt).run(location='testsweep', overwrite=True)
    while data.sync():
        time.sleep(0.1)

DataSet: DataMode.PULL_FROM_SERVER, location='testsweep'
   volt_set: volt
   volt_0: volt
   volt_1: volt
started at 2016-04-21 15:51:17
[Time Loop 2]
Elapsed: 1.569089651107788
